# Notebook for preprocessing the LOFAR catalogues before XID+

it outputs a modified LOFAR radio data table with two new columns. the first column is XID+_rerun and is a boolean array stating whether a source should ahve XID+ rerun or not. The second is a string column that says what where the fir for that source comes from.

In [1]:
from astropy.io import ascii, fits
import astropy
import pylab as plt
%matplotlib inline
from astropy import wcs
from astropy.table import Table,Column,join,hstack,vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
import pymoc
import glob
from time import sleep
import os
import pymoc
import mocpy

import numpy as np
import xidplus
from xidplus import moc_routines
import pickle
import xidplus.catalogue as cat

import sys
#from herschelhelp_internal.utils import inMoc,flux_to_mag
from xidplus.stan_fit import SPIRE

import aplpy
import seaborn as sns
#sns.set(color_codes=True)
import pandas as pd
#sns.set_style("white")
import xidplus.posterior_maps as postmaps
#from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
import pyvo as vo
#from herschelhelp_internal.utils import inMoc
from pymoc.io.fits import read_moc_fits



In [7]:
lofar = Table.read('data/EN1_opt_spitzer_merged_vac_opt3as_irac4as_all_hpx_public.fits')

mask = ~np.isnan(lofar['F_SPIRE_250'])
c = SkyCoord(lofar['RA'][mask],lofar['DEC'][mask],unit='deg')
spire_moc = pymoc.util.catalog.catalog_to_moc(c,60,15)
spire_moc = pymoc.moc.MOC()
read_moc_fits(spire_moc,'/lustre/scratch/astro/im281/HELP/dmu_products/dmu2/dmu2_field_coverages/ELAIS-N1_MOC.fits')
print('SPIRE covers: {}sq degrees'.format(spire_moc.area_sq_deg))

mask = ~np.isnan(lofar['F_PACS_100'])
c = SkyCoord(lofar['RA'][mask],lofar['DEC'][mask],unit='deg')
pacs_moc = pymoc.util.catalog.catalog_to_moc(c,60,15)
print('PACS covers: {}sq degrees'.format(pacs_moc.area_sq_deg))

mask = ~np.isnan(lofar['F_MIPS_24'])
c = SkyCoord(lofar['RA'][mask],lofar['DEC'][mask],unit='deg')
mips_moc = pymoc.util.catalog.catalog_to_moc(c,60,15)
print('MIPS covers: {}sq degrees'.format(mips_moc.area_sq_deg))

SPIRE covers: 13.507484555454763sq degrees
PACS covers: 9.677919150573207sq degrees
MIPS covers: 9.72350746163187sq degrees


In [28]:
imfolder='../../../../../HELP/dmu_products/dmu17/dmu17_HELP_Legacy_maps/ELAIS-N1/data/'

pswfits=imfolder+'wp4_elais-n1_mips24_map_v1.0.fits.gz'#SPIRE 250 map

MIPS_Map = fits.open(pswfits)

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header
mips_moc = mocpy.MOC.from_image(im250hdu,15)

mips_moc.write('data/data_release/final_mocs/mips_moc_en1.fits',format='fits',overwrite=True)

mips_moc = pymoc.moc.MOC()
read_moc_fits(mips_moc,'data/data_release/final_mocs/mips_moc_en1.fits')
print('SPIRE covers: {}sq degrees'.format(mips_moc.area_sq_deg))

SPIRE covers: 13.834475593100896sq degrees


In [29]:
imfolder='../../../../../HELP/dmu_products/dmu18/dmu18_ELAIS-N1/data/'

im100fits=imfolder + 'ELAIS-N1-100um-img_wgls.fits'#PACS 100 map
#-----100-------------
hdulist = fits.open(im100fits)
im100phdu=hdulist[0].header
im100hdu=hdulist[0].header
pacs_moc = mocpy.MOC.from_image(im100hdu,15)

pacs_moc.write('data/data_release/final_mocs/pacs_moc_en1.fits',format='fits',overwrite=True)

pacs_moc = pymoc.moc.MOC()
read_moc_fits(pacs_moc,'data/data_release/final_mocs/pacs_moc_en1.fits')
print('SPIRE covers: {}sq degrees'.format(pacs_moc.area_sq_deg))

SPIRE covers: 24.23288471879231sq degrees


In [27]:
imfolder='../../../../../HELP/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/'

pswfits=imfolder+'ELAIS-N1_SPIRE250_v1.0.fits'#SPIRE 250 map

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist['image'].header
spire_moc = mocpy.MOC.from_image(im250hdu,15)

spire_moc.write('data/data_release/final_mocs/spire_moc_en1.fits',format='fits',overwrite=True)

spire_moc = pymoc.moc.MOC()
read_moc_fits(spire_moc,'data/data_release/final_mocs/spire_moc_en1.fits')
print('SPIRE covers: {}sq degrees'.format(spire_moc.area_sq_deg))

SPIRE covers: 24.978424412691524sq degrees


SPIRE covers: 24.978424412691524sq degrees


# spire_moc

In [3]:
#lofar01 = Table.read('data/data_release/final_cross_match_catalogue-v0.1.fits')
#lofar05 = Table.read('data/data_release/final_cross_match_catalogue-v0.5.fits')
#new_x_old = join(lofar05,lofar01,join_type='left',keys='Source_Name')

In [17]:
#mask = new_x_old['optRA_1']==new_x_old['optRA_2']
lofar = Table.read('data/data_release/final_cross_match_catalogue-v0.5.fits')

In [3]:
#Read in the LOFAR data and select sources with no FIR measurements from XID+
lofar = Table.read('data/data_release/final_cross_match_catalogue-v0.5.fits')
mask = (~np.isnan(lofar['F_SPIRE_250'])) | (~np.isnan(lofar['F_SPIRE_350'])) | (~np.isnan(lofar['F_SPIRE_500']))
lofar = lofar[~mask]

ras = lofar['optRA']
mask = np.isnan(ras)
print(np.sum(mask))
ras[mask] = lofar['RA'][mask]

decs = lofar['optDec']
mask = np.isnan(decs)
print(np.sum(mask))
decs[mask] = lofar['DEC'][mask]

ids = lofar['Source_Name']

807
807


In [4]:
len(lofar)

8370

In [5]:
#Read in the HELP masterlist and select the wanted columns
columns = 'ra','dec','help_id','flag_optnir_det','f_mips_24'
masterlist = Table.read('../../../../../HELP/dmu_products/dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20171020.fits')
help_masterlist = masterlist[columns]

In [6]:
mips_mask = (help_masterlist['flag_optnir_det']>=5) & (help_masterlist['f_mips_24']>20)

prior_cat = help_masterlist[mips_mask]
print('number of help prior sources is: {}'.format(len(prior_cat)))

number of help prior sources is: 269611


In [7]:
cra = Column(ras,name='ra')
cdec = Column(decs,name='dec')
cids = Column(ids,name='help_id')
cdet = Column(np.zeros(len(ras))-99,name='flag_optnir_det')
cmips = Column(np.zeros(len(ras))*np.nan,name='f_mips_24')
lofar_prior = Table()
lofar_prior.add_columns([cra,cdec,cids,cdet,cmips])

In [8]:
prior_cat = vstack([prior_cat,lofar_prior])
len(prior_cat)

277981

In [9]:
Table.write(prior_cat,'data/data_release/xidplus_prior_cat.fits',overwrite=True)

### Create a seperate prior list for rerunning all sources even if they are nearby a prior list source

In [102]:
lofar = Table.read('data/data_release/final_cross_match_catalogue-v0.1.fits')

ras = lofar['optRA']
mask = np.isnan(ras)
print(np.sum(mask))
ras[mask] = lofar['RA'][mask]

decs = lofar['optDec']
mask = np.isnan(decs)
print(np.sum(mask))
decs[mask] = lofar['DEC'][mask]

ids = lofar['Source_Name']
print(len(ras))

1236
1236
31748


In [103]:
columns = 'ra','dec','help_id','flag_optnir_det','f_mips_24'
masterlist = Table.read('../../../../../HELP/dmu_products/dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20171020.fits')
help_masterlist = masterlist[columns]

In [104]:
mips_mask = (help_masterlist['flag_optnir_det']>=5) & (help_masterlist['f_mips_24']>20)

prior_cat = help_masterlist[mips_mask]
print('number of help prior sources is: {}'.format(len(prior_cat)))

number of help prior sources is: 269611


In [105]:
cra = Column(ras,name='ra')
cdec = Column(decs,name='dec')
cids = Column(ids,name='help_id')
cdet = Column(np.zeros(len(ras))-99,name='flag_optnir_det')
cmips = Column(np.zeros(len(ras))*np.nan,name='f_mips_24')
lofar_prior = Table()
lofar_prior.add_columns([cra,cdec,cids,cdet,cmips])

In [106]:
prior_cat = vstack([prior_cat,lofar_prior])
len(prior_cat)

301359

In [107]:
Table.write(prior_cat,'data/data_release/xidplus_prior_cat_rerun_all.fits',overwrite=True)

## Create a prior list for the QSO sources

In [4]:
lofar = Table.read('../LOFAR_QSO/data/SDSSrcQSO_LOFARnondetect.fits')

ras = lofar['RA']
decs = lofar['DEC']
ids = lofar['SDSS_NAME']
lofar[0]

SDSS_NAME,RA,DEC,THING_ID,PLATE,MJD,FIBERID,SPECTRO,Z,Z_ERR,SOURCE_Z,Z_VI,Z_PIPE,Z_PIPE_ERR,ZWARNING,Z_PCA,Z_PCA_ER,Z_MGII,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2,N_SPEC_SDSS,N_SPEC_BOSS,N_SPEC,PLATE_DUPLICATE [51],MJD_DUPLICATE [51],FIBERID_DUPLICATE [51],SPECTRO_DUPLICATE [51],BI_CIV,ERR_BI_CIV,RUN_NUMBER,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],MI,GAL_EXT [5],RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,FLUX_0.2_2.0keV,FLUX_0.2_2.0keV_ERR,FLUX_2.0_12.0keV,FLUX_2.0_12.0keV_ERR,FLUX_0.2_12.0keV,FLUX_0.2_12.0keV_ERR,LUM_0.2_12.0keV,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAGS,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,DR7_index,L6mu,L6mu_err,sample
-,deg,deg,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,km / s,km / s,-,-,-,-,-,nanomaggies,-,mag,mag,-,mag,counts/s,-,arcsec,erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / s,arcsec,-,-,-,-,-,Vega,Vega,-,-,Vega,Vega,-,-,Vega,Vega,-,-,arcsec,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,arcsec,-,W / (Hz m2),W / (Hz m2),W / (Hz m2),W / (Hz m2),W / (Hz m2),W / (Hz m2),W / (Hz m2),W / (Hz m2),-,mJy,-,arcsec,,,,
bytes18,float64,float64,int32,int16,int32,int16,bytes4,float64,float64,bytes4,float64,float64,float64,int16,float64,float64,float64,int64,int64,int64,int64,int64,int64,int16,int16,int16,int32,int32,int32,bytes1,float64,float64,int16,bytes1,int16,int16,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float32,int16,float64,float64,float32,int16,float64,float64,float32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,bytes4,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,int32,float64,float64,bytes4
142640.66+321151.9,216.66943359375,32.197757720947266,-1,1841,53491,317,SDSS,0.2019,0.0,DR7Q,-1.0,-1.0,-1.0,-1,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,-1 .. -1,-1 .. -1,-1 .. -1,- .. -,0.0,0.0,0,-,0,0,-,-9999.0 .. -9999.0,-9999.0 .. -9999.0,18.825000762939453 .. 17.885000228881836,0.023000000044703484 .. 0.028999999165534973,-22.478964,0.06 .. 0.01722,0.0,0.0,-1.0,nan,nan,nan,nan,nan,nan,nan,nan,1.0,6.457155227661133,1.5975519690476745,15.947311401367188,294.5100303885923,16.269000000000002,0.111,10.1,2,15.091,0.093,12.8,2,14.009,0.052000000000000005,24.1,2,0.297782,12.93,0.024,46.0,1.216,11.822,0.021,52.099998474121094,1.213,8.969,0.026,42.400001525878906,0.8701,6.419,0.024,23.0,1.031,0000,AAAA,0.045578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.0,-1.0,-1.0,77651,44.393281024044946,0.013368619973654723,cQSO


In [5]:
columns = 'ra','dec','help_id','flag_optnir_det','f_mips_24'
masterlist = Table.read('../../../../../HELP/dmu_products/dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20171020.fits')
help_masterlist = masterlist[columns]

In [6]:
mips_mask = (help_masterlist['flag_optnir_det']>=5) & (help_masterlist['f_mips_24']>20)

prior_cat = help_masterlist[mips_mask]
print('number of help prior sources is: {}'.format(len(prior_cat)))

number of help prior sources is: 269611


In [7]:
cra = Column(ras,name='ra')
cdec = Column(decs,name='dec')
cids = Column(ids,name='help_id')
cdet = Column(np.zeros(len(ras))-99,name='flag_optnir_det')
cmips = Column(np.zeros(len(ras))*np.nan,name='f_mips_24')
lofar_prior = Table()
lofar_prior.add_columns([cra,cdec,cids,cdet,cmips])

In [8]:
prior_cat = vstack([prior_cat,lofar_prior])
len(prior_cat)

270374

In [10]:
#Table.write(prior_cat,'data/data_release/xidplus_prior_cat_QSO_Calistro.fits',overwrite=True)
from astropy.io import registry
from astropy.table.info import serialize_method_as

with serialize_method_as(prior_cat, None):
            registry.write(prior_cat,'data/xidplus_prior_cat_QSO_Calistro.fits',format='fits')

<class 'astropy.table.table.Table'>
fits
